# Setup


In [1]:
from imports import *  

# Plotting Functions

In [2]:
# Plotting functions
def plot_network_degree_distribution(G, directed=True, title='title'):
    if directed:
        degrees = np.array([degree for node, degree in G.out_degree()])
    else:
        degrees = np.array([degree for node, degree in G.degree()])
    # Create the histogram with a KDE
    sns.set(style="whitegrid")
    plt.figure(figsize=(8, 6))
    sns.histplot(degrees, kde=False, bins=150, stat="count")
    # Calculate the mean
    mean_value = np.mean(degrees)
    print(mean_value)
    print(np.median(degrees))

    # Plot a vertical line at the mean value
    plt.axvline(mean_value, color='b', linestyle='--', linewidth=2)
    plt.text(mean_value + 0.1, plt.ylim()[1] * 0.9, f'Mean: {mean_value}', color='b')
    # plt.text(mean_value + 0.1, plt.ylim()[1] * 0.9, 'Mean: {:.2f}'.format(mean_value), color='b')

    plt.title('Timeline Smooth Histogram for: ' + title)
    plt.xlabel('Degree')
    plt.ylabel('Count')
    plt.xticks(fontsize=8,rotation=20)
    plt.show()
    
def plot_loglog(G,directed=True,m=10):
    if directed:
        # Get the in-degree of all nodes
        out_degrees = [d for _, d in G.out_degree()]

        # Compute the histogram
        max_degree = max(out_degrees)
        degree_freq = [out_degrees.count(i) for i in range(max_degree + 1)]
    else:
        degree_freq = nx.degree_histogram(G)
    degrees = range(len(degree_freq))
    plt.figure(figsize=(12, 8))
    plt.loglog(degrees[m:], degree_freq[m:],'go-')
    plt.xlabel('Degree')
    plt.ylabel('Frequency')
    plt.title('Log-Log plot of the degree distribution')
    
    
    def scatter_plot(df, target_variable="share_of_correct_agents_at_convergence"):
        # Select numerical columns excluding unique ID and target variable
        numerical_columns = df.select_dtypes(include=["number"]).columns.tolist()
        numerical_columns.remove(target_variable)  # Remove target variable from independent variables

        # Generate scatter plots for each numerical column against the target variable
        num_plots = len(numerical_columns)
        fig, axes = plt.subplots(nrows=(num_plots + 1) // 2, ncols=2, figsize=(10, num_plots * 2))
        axes = axes.flatten()

        for i, column in enumerate(numerical_columns):
            axes[i].scatter(df[column], df[target_variable], alpha=0.5)
            axes[i].set_xlabel(column)
            axes[i].set_ylabel(target_variable)
            axes[i].set_title(f"{column} vs {target_variable}")
            axes[i].grid(True)

        # Hide any unused subplots
        for j in range(i + 1, len(axes)):
            fig.delaxes(axes[j])

        plt.tight_layout()
        plt.show()

# Network Statistics

In [3]:
# Network statistics
def calculate_degree_gini(G, directed = True):
    if directed:
        degrees = [deg for _, deg in G.out_degree()]
    else:
        degrees = [deg for _, deg in G.degree()]
    # Sort the degrees in ascending order
    sorted_x = np.sort(np.array(degrees))
    n = len(np.array(degrees))
    cumx = np.cumsum(sorted_x, dtype=float)
    gini = (n + 1 - 2 * np.sum(cumx) / cumx[-1]) / n

    return gini

def find_reachability_dominator_set(G):
    """
    Finds a minimal reachability dominator set in a directed graph G.

    Parameters:
        G (nx.DiGraph): A directed graph.

    Returns:
        set: A set of nodes A such that every node in G is reachable from some node in A.
    """
    # Step 1: Compute strongly connected components
    sccs = list(nx.strongly_connected_components(G))

    # Step 2: Build the condensation graph
    C = nx.condensation(G, sccs)

    # Step 3: Find source SCCs (no incoming edges)
    source_sccs = [node for node in C.nodes if C.in_degree(node) == 0]

    # Step 4: Pick one representative node from each source SCC
    reachability_dominator_set = set()
    scc_list = C.graph['mapping']  # maps node -> scc index
    inverse_scc_map = {}
    for node, scc_id in scc_list.items():
        inverse_scc_map.setdefault(scc_id, []).append(node)

    for source_scc in source_sccs:
        representative = inverse_scc_map[source_scc][0]  # pick one node from this SCC
        reachability_dominator_set.add(representative)

    return len(reachability_dominator_set), len(reachability_dominator_set)/len(G), len(C), len(C)/len(G)

def network_statistics(G, directed = True):
    stats = {}

    # Average degree
    if directed:
        degrees = [deg for _, deg in G.out_degree()]
    else:
        degrees = [deg for _, deg in G.degree()]
    stats['average_degree'] = sum(degrees) / len(degrees)

    # Gini coefficient
    #print(degrees)
    stats['degree_gini_coefficient'] = calculate_degree_gini(G, directed=directed)

    # Compute clustering for each node
    # it allows us to use weights, which we neglect...
    clustering_values = nx.clustering(G)
    # Compute the average clustering coefficient manually
    average_clustering = sum(clustering_values.values()) / len(clustering_values)
    stats['approx_average_clustering_coefficient'] = average_clustering

    if directed:    
        if nx.is_strongly_connected(G):
            stats['avg_path_length'] = nx.average_shortest_path_length(G)
        else:
            stats['avg_path_length'] = len(G.nodes)+1
            # largest_component = max(nx.weakly_connected_components(G), key=len)
            # subgraph = G.subgraph(largest_component)
            # stats['diameter'] = nx.diameter(subgraph)
    else:
        if nx.is_connected(G):
            stats['avg_path_length'] = nx.average_shortest_path_length(G)
        else:
            stats['avg_path_length'] = len(G.nodes)+1
            # largest_component = max(nx.connected_components(G), key=len)
            # subgraph = G.subgraph(largest_component)
            # stats['diameter'] = nx.diameter(subgraph)

    if directed:
        out_degrees = np.array([d for _, d in G.out_degree()])
        # out_degrees = np.array([d for _, d in graph.out_degree()])
        in_hist, _ = np.histogram(out_degrees, bins=range(np.max(out_degrees) + 2), density=True)
        # out_hist, _ = np.histogram(out_degrees, bins=range(np.max(out_degrees) + 2), density=True)
        out_entropy = -np.sum(in_hist[in_hist > 0] * np.log(in_hist[in_hist > 0]))
        # out_entropy = -np.sum(out_hist[out_hist > 0] * np.log(out_hist[out_hist > 0]))
        stats['degree_entropy'] = out_entropy
    else:
        degrees = np.array([d for _, d in G.degree()])
        hist, _ = np.histogram(degrees, bins=range(np.max(degrees) + 2), density=True)
        entropy = -np.sum(hist[hist > 0] * np.log(hist[hist > 0]))
        stats['degree_entropy'] = entropy

    # Add additional metrics as needed here, e.g., centrality measures
    stats['reachability_dominator_set_size'] = find_reachability_dominator_set(G)[0]
    stats['reachability_dominator_set_ratio'] = find_reachability_dominator_set(G)[1]
    stats['condensation_graph_size'] = find_reachability_dominator_set(G)[2]
    stats['condensation_graph_ratio'] = find_reachability_dominator_set(G)[3]
    return stats

# Variation Methods

Main contribution of the paper:

- Methodological contribution of empirical robustness for _network_ epistemology. 

Key challenge:

- Test whether inequality (or some other network feature such as density, clustering, or diameter) increases reliability (or some other metric such as speed).
- Counterfactual: had the network been more equal, the group would have been more reliable.
- This requires us to identify networks that differ in their inequality but are otherwise maximally similar.
- We consider three network features: density, clustering, and degree inequality. So networks that have the similar density, clustering and degree inequality are considered maximally similar. (I left out diameter here.)

To achieve this, we develop some variation methods. The main goals for these variation methods are:

1. Control: Ability to tinker with specific network features (density, inequality, clustering)
2. Simplicity
3. Computational tractability
4. Link to (individualistic) intervention 

There are two types of variation methods: 

1. Possibly complex variation methods that can be used to produce networks with specific network properties (density, inequality, clustering), i.e., high control
2. Intuitive and simple variation methods that yield a lower degree of control over the specific network properties (density, inequality, clustering)

The basic ideas in these variation methods are the following:

1. Density
    - Increase by adding edges
    - Keep constant by rewiring edges
    - Decrease by removing edges [not implemented]
2. Clustering
    - Note: the local clustering coefficient of a given node basically is the number of triangles that pass by that node divided by the number of possible triangles that pass by that node. 
    - Increase by adding edges that create new triangles
    - Keep constant ???
    - Decrease by removing edges from existing triangles
3. Inequality
    - Decrease by adding edges randomly (following the uniform degree distribution)
    - Keep constant by adding edges following the original degree distribution (i.e., that of the PUD network)
    - Increase by (a) sequentially adding edges preferentially, or  (b) by adding edges following a degree distribution that is more unequal than the original one [neither implemented]

## Hop over to Outcomes section to see the results of the different variation methods

## Simple variation methods

### Equalizers: tinker with inequality

1. `randomize_network`: Randomly rewire edges (following the uniform degree distribution) [THIS IS WHAT WE CURRENTLY DO]
    - Density $=$
    - Clustering $\downarrow$
    - Inequality $\downarrow$
2. `equalize`: Rewire triangles: Take a triangle, take a node in the triangle, remove the edge in the triangle that does not contain the node, then add a random new edge that creates a new triangle that passed by the node. (The goal was to keep clustering somewhat equal, while increasing equality.)
    - Density $=$
    - Clustering $\approx \downarrow$
    - Inequality $\downarrow$

### Densify
1. [not implemented] Randomly add edges (following the uniform degree distribution)
    - Expectations
        - Density $\uparrow$
        - Clustering $\downarrow$
        - Inequality $\downarrow$
2. `densify`: Add edges following the original degree distribution
    - Density $\uparrow$
    - Clustering $\downarrow$
    - Inequality $=$
3. `cluster`: Add edges that create new triangles (taking into account the original degree distribution)
    - Density $\uparrow$
    - Clustering $\Uparrow$
    - Inequality $\approx$ 

### Clustering
1. `decluster`: Remove edges from existing triangles and add new edge following the original degree distribution
    - Density $=$
    - Clustering $\Downarrow$
    - Inequality $\approx\downarrow$


## Complex variation methods

1. `densify_fancy`: Add edges in such a way to attempt to reach a target clustering coefficient and a target degree distribution (original or uniform). 
    - Basically, the algorithm checks whether the target clustering coefficient has been reached. If not, it adds an edge that increases clustering. If yes, it adds a new edge following the target degree distribution. 
    - **Computationally costly**: after an edge is added, the algorithm calculates the clustering coefficient of the new network, which is computationally costly to an unacceptable degree (I think). 
    - **High degree of control**, especially regarandoming the clustering coefficient: For example, can be used to achieve the following:
        1. Only increase density
            - Density $\uparrow$
            - Clustering $=$
            - Inequality $=$
        2. Increase density and decrease inequality
            - Density $\uparrow$
            - Clustering $=$
            - Inequality $\downarrow$
2. `densify_fancy_speed_up`: This method basically works the same as `densify_fancy`, but has a significant speed up. Instead of calculating the average clustering coefficient whenever an edge is added, we only calculate the new local clusterinf coefficient for nodes that are affected by the newly added edge. 
    - **Computationally not cheap**: although the algorithm is much quicker than `densify_fancy`, it still takes some time. I expect that the computational costs are acceptable.
    - **High control**, especially regarandoming the clustering coefficient.
3. `densify_semi_fancy`: Add edges either to increase clustering or following the target degree distribution — with a fixed probability.
    - Basically, the algorithm throws a (biased) coin to determine whether it will add an edge that increases clustering or add one following the target degree distribution. 
    - Computationally fast. Relies on a biased coin toss instead of calculating the clustering coefficient. The drawback is that the clustering coefficient will change to some degree.
    - Pretty high degree of control.


- Note: we could keep the density fixed by first removing a number of edges and then using `densify_fancy` or `densify_semi_fancy` to add the same number of edges. 

## Thoughts and observations

- I was surprised to learn that both the in-degree and the out-degree distributions of the PUD network are scale free. However, they are not neatly correlated. Nonetheless, my suspicion is that there is a pattern between an agent’s number of publication and both its in-degree and its out-degree. 
- I was surprised to learn that the clustering coefficient is not based on directed graphs (`nx.average_clustering`). 
- In a sense, we have currently only implemented the target degree distribution of the degree distribution of the input network (the PUD network) or the uniform distribution. We could consider other degree distributions (i.e., an unequal distribution that flips the original degree distribution)
- 

## Helper Functions

In [4]:
def get_triangles(net: nx.DiGraph):
    """Return the list of all triangles in a directed graph G."""
    triangles = []
    for clique in nx.enumerate_all_cliques(net.to_undirected()):
        if len(clique) <= 3:
            if len(clique) == 3:
                triangles.append(clique)
        else:
            return triangles
    return triangles

## Randomization

In [5]:
def randomize_network(G, p_rewiring):
    # Check if the graph is directed
    is_directed = G.is_directed()

    # Get edges and nodes
    edges = list(G.edges()).copy()
    random.shuffle(edges)
    edges_set = set(edges)
    new_edges_set = edges_set.copy()
    nodes = list(G.nodes()).copy()

    # Find which edges to remove
    to_remove_set = set()
    for old_edge in edges:
        if random.random() < p_rewiring:  # p probability to rewire an edge
            to_remove_set.add(old_edge)
            new_edges_set.remove(old_edge)

    # Generate a new edges
    for edge in to_remove_set:
        new_edge = (random.choice(nodes), random.choice(nodes))
        if not is_directed:
            new_edge = tuple(sorted(new_edge))  # Ensure (u, v) == (v, u) for undirected graphs

        # Avoid duplicate edges and self-loops
        while (new_edge in new_edges_set) or (new_edge[0] == new_edge[1]):
            new_edge = (random.choice(nodes), random.choice(nodes))
            if not is_directed:
                new_edge = tuple(sorted(new_edge))

        new_edges_set.add(new_edge)

    # Create a new graph with updated edges
    G_new = G.copy()
    G_new.remove_edges_from(to_remove_set)
    G_new.add_edges_from(new_edges_set)

    return G_new

## Equalize

In [6]:
def equalize(net: nx.DiGraph, n: int) -> nx.DiGraph:
    """
    Equalize the network by rewiring n random edges.
    """
    equalized_net = copy.deepcopy(net)
    triangles = get_triangles(net)
    rewired_triangles = random.sample(triangles, n)
    
    for triangle in rewired_triangles:
        edge = triangle[-2:]  # Take the last two nodes as the edge to be rewired
        # Remove edge
        if equalized_net.has_edge(*edge):
            equalized_net.remove_edge(*edge)
        elif equalized_net.has_edge(edge[1], edge[0]):
            equalized_net.remove_edge(edge[1], edge[0]) 
        else:
            continue
        
        # Add new edge to create a new triangle that passes by the first node
        node = triangle[0] 
        neighbors = list(net.predecessors(node)) + list(net.successors(node))

        sources_sample = random.choices(neighbors, k=10)
        targets_sample = random.choices(neighbors, k=10)
        edge_sample = [
            (source, target) 
            for source in sources_sample 
            for target in targets_sample 
            if source != target and not equalized_net.has_edge(source, target)]
        new_edge = random.choice(edge_sample) # Throws an error if no edges are available
        equalized_net.add_edge(*new_edge)
    return equalized_net

## Densify

In [7]:
def densify_network(net: nx.DiGraph, n_edges: int) -> nx.DiGraph:
    # Create a copy of the original network
    densified_net = copy.deepcopy(net)
    
    # Get the degree distribution
    in_degrees = dict(net.in_degree())
    out_degrees = dict(net.out_degree())
    multiplier = 10
    targets = random.choices(
        list(in_degrees.keys()), 
        weights=list(in_degrees.values()), 
        k=multiplier*n_edges
    )
    sources = random.choices(
        list(out_degrees.keys()), 
        weights=out_degrees.values(), 
        k=multiplier*n_edges
    )

    edges_new = list(set(zip(sources, targets))) 
    edges_new = [edge for edge in edges_new if edge[0] != edge[1]]  
    edges_new = [edge for edge in edges_new if not edge in net.edges()]
    edges_new = edges_new[:n_edges] 
    
    densified_net.add_edges_from(edges_new)
    return densified_net

In [8]:
def densify_semi_fancy(
    net: nx.DiGraph, n_edges: int, p_increase_clustering: float, target_degree_dist: str = "original",
) -> nx.DiGraph:
    """
    Densifies a directed network by adding new edges, balancing between increasing 
    ing
    and preserving a target degree distribution.

    Parameters
    ----------
    net : nx.DiGraph
        The original directed network to densify.
    n_edges : int
        The number of new edges to add.
    p_increase_clustering : float
        Probability (between 0 and 1) that a new edge is added to increase clustering
        (i.e., create new triangles). Otherwise, new edges are added based on the 
        target degree distribution.
    target_degree_dist : str, optional
        The target degree distribution for new edges.
        "original" uses the original network's degree distribution,
        "uniform" assigns equal probability to all nodes. Default is "original".

    Returns
    -------
    nx.DiGraph
        A new directed network with additional edges.
    """
    
    # Create a copy of the original network
    net_new = copy.deepcopy(net)
    
    if target_degree_dist == "original":
        # Use the original degree distribution
        out_degrees = dict(net.out_degree())
        in_degrees = dict(net.in_degree())
    if target_degree_dist == "uniform":
        out_degrees = {node: 1 for node in net.nodes()}
        in_degrees = {node: 1 for node in net.nodes()}

    # Add edges in neighborhoods
    n_edges_added = 0
    edges_added_clustering = 0
    edges_added_degree_dist = 0
    while n_edges_added < n_edges:
        if random.random() < p_increase_clustering:
            # Add new edge to increase clustering
            possible_edges = []
            while possible_edges == []:
                node = random.choice(list(net.nodes()))
                neighbors = list(net.predecessors(node)) + list(net.successors(node))
                out_degrees_neighbors = {node: out_degrees[node] for node in neighbors}
                in_degrees_neighbors = {node: in_degrees[node] for node in neighbors}
                out_weights = out_degrees_neighbors.values()
                if all(out_weights) == 0:
                    out_weights = np.ones(len(out_degrees_neighbors.keys()))
                
                in_weights = in_degrees_neighbors.values()
                if all(in_weights) == 0:
                    in_weights = np.ones(len(in_degrees_neighbors.keys()))
                
                sources = random.choices(list(out_degrees_neighbors.keys()), weights=out_weights, k=10)
                targets = random.choices(list(in_degrees_neighbors.keys()), weights=in_weights, k=10)
                possible_edges = [
                    (source, target) for source in sources for target in targets
                    if source != target and not net_new.in_edges(source, target)
                ]
                if possible_edges != []:
                    new_edge = random.choice(possible_edges)
                    n_edges_added += 1
                    net_new.add_edge(*new_edge)
                    edges_added_clustering += 1
        else:
            # Add new edge based on target degree distribution
            edge_sample = []
            while edge_sample == []:
                sources_sample = random.choices(list(out_degrees.keys()), weights=out_degrees.values(), k=10)
                targets_sample = random.choices(list(in_degrees.keys()), weights=in_degrees.values(), k=10)
                edge_sample = [
                    (source, target) 
                    for source in sources_sample 
                    for target in targets_sample 
                    if source != target and not net_new.has_edge(source, target)]
                if edge_sample != []:
                    new_edge = random.choice(edge_sample) # Throws an error if no edges are available
                    n_edges_added += 1
                    net_new.add_edge(*new_edge)
                    edges_added_degree_dist += 1
        # print(f"{n_edges_added=:,} edges added")
    print(f"{edges_added_clustering:,} edges added to increase clustering")
    print(f"{edges_added_degree_dist:,} edges added based on {target_degree_dist} degree distribution")
    return net_new

In [9]:
def densify_fancy(
    net: nx.DiGraph, n_edges: int, target_degree_dist: str = "original", target_clustering: float = None,
) -> nx.DiGraph:
    """
    Densifies a directed network by adding new edges to increase its density, 
    while optionally targeting a specific degree distribution and clustering coefficient.
    Priority is given to targeting the specified clustering coefficient.

    Parameters
    ----------
    net : nx.DiGraph
        The original directed network to densify.
    n_edges : int
        The number of edges to add.
    target_degree_dist : str, optional
        The target degree distribution for new edges. 
        "original" preserves the original degree distribution, 
        "uniform" assigns equal probability to all nodes. Default is "original".
    target_clustering : float, optional
        The desired average clustering coefficient. If None, uses the original network's clustering.

    Returns
    -------
    nx.DiGraph
        A new directed network with increased density and optionally modified clustering/degree distribution.
    """
    
    # Create a copy of the original network
    net_new = copy.deepcopy(net)
    
    if target_clustering is None:
        target_clustering = nx.average_clustering(net)
    if target_degree_dist == "original":
        # Use the original degree distribution
        out_degrees = dict(net.out_degree())
        in_degrees = dict(net.in_degree())
    if target_degree_dist == "uniform":
        out_degrees = {node: 1 for node in net.nodes()}
        in_degrees = {node: 1 for node in net.nodes()}

    # Add edges in neighborhoods
    n_edges_added = 0
    edges_added_clustering = 0
    edges_added_degree_dist = 0
    new_clustering = nx.average_clustering(net_new)
    while n_edges_added < n_edges:
        if new_clustering < target_clustering:
            # Add new edge to increase clustering
            node = random.choice(list(net.nodes()))
            neighbors = list(net.predecessors(node)) + list(net.successors(node))
            out_degrees_neighbors = {node: out_degrees[node] for node in neighbors}
            in_degrees_neighbors = {node: in_degrees[node] for node in neighbors}
            out_weights = out_degrees_neighbors.values()
            if all(out_weights) == 0:
                out_weights = np.ones(len(out_degrees_neighbors.keys()))
            in_weights = in_degrees_neighbors.values()
        
            if all(in_weights) == 0:
                in_weights = np.ones(len(in_degrees_neighbors.keys()))
            
            sources = random.choices(list(out_degrees_neighbors.keys()), weights=out_weights, k=10)
            targets = random.choices(list(in_degrees_neighbors.keys()), weights=in_weights, k=10)
            possible_edges = [
                (source, target) for source in sources for target in targets
                if source != target and not net_new.in_edges(source, target)
            ]
            if possible_edges != []:
                new_edge = random.choice(possible_edges)
                n_edges_added += 1
                net_new.add_edge(*new_edge)
                new_clustering = nx.average_clustering(net_new)
                edges_added_clustering += 1
        else:
            # Add new edge based on target degree distribution
            sources_sample = random.choices(list(out_degrees.keys()), weights=out_degrees.values(), k=10)
            targets_sample = random.choices(list(in_degrees.keys()), weights=in_degrees.values(), k=10)
            edge_sample = [
                (source, target) 
                for source in sources_sample 
                for target in targets_sample 
                if source != target and not net_new.has_edge(source, target)]
            if edge_sample != []:
                new_edge = random.choice(edge_sample) # Throws an error if no edges are available
                n_edges_added += 1
                net_new.add_edge(*new_edge)
                new_clustering = nx.average_clustering(net_new)
                edges_added_degree_dist += 1
        print(f"{n_edges_added=:,} edges added")
    print(f"{edges_added_clustering:,} edges added to increase clustering")
    print(f"{edges_added_degree_dist:,} edges added based on {target_degree_dist} degree distribution")
    return net_new

In [10]:
def densify_fancy_speed_up(
    net: nx.DiGraph, n_edges: int, target_degree_dist: str = "original", target_average_clustering: float = None,
) -> nx.DiGraph:
    """
    Densifies a directed network by adding new edges to increase its density, 
    while optionally targeting a specific degree distribution and clustering coefficient.
    Priority is given to targeting the specified clustering coefficient.

    Parameters
    ----------
    net : nx.DiGraph
        The original directed network to densify.
    n_edges : int
        The number of edges to add.
    target_degree_dist : str, optional
        The target degree distribution for new edges. 
        "original" preserves the original degree distribution, 
        "uniform" assigns equal probability to all nodes. Default is "original".
    target_clustering : float, optional
        The desired average clustering coefficient. If None, uses the original network's clustering.

    Returns
    -------
    nx.DiGraph
        A new directed network with increased density and optionally modified clustering/degree distribution.
    """
    
    # Create a copy of the original network
    net_new = copy.deepcopy(net)
    
    if target_average_clustering is None:
        target_average_clustering = nx.average_clustering(net)
    if target_degree_dist == "original":
        # Use the original degree distribution
        out_degrees = dict(net.out_degree())
        in_degrees = dict(net.in_degree())
    if target_degree_dist == "uniform":
        out_degrees = {node: 1 for node in net.nodes()}
        in_degrees = {node: 1 for node in net.nodes()}
    clustering_dict: dict = nx.clustering(net_new)
    
    # Add edges in neighborhoods
    n_edges_added = 0
    edges_added_clustering = 0
    edges_added_degree_dist = 0
    new_average_clustering = np.average(list(clustering_dict.values()))
    while n_edges_added < n_edges:
        if new_average_clustering < target_average_clustering:
            # Add new edge to increase clustering
            node = random.choice(list(net.nodes()))
            neighbors = list(net.predecessors(node)) + list(net.successors(node))
            out_degrees_neighbors = {node: out_degrees[node] for node in neighbors}
            in_degrees_neighbors = {node: in_degrees[node] for node in neighbors}
            out_weights = out_degrees_neighbors.values()
            if all(out_weights) == 0:
                out_weights = np.ones(len(out_degrees_neighbors.keys()))
            in_weights = in_degrees_neighbors.values()
        
            if all(in_weights) == 0:
                in_weights = np.ones(len(in_degrees_neighbors.keys()))
            
            sources = random.choices(list(out_degrees_neighbors.keys()), weights=out_weights, k=10)
            targets = random.choices(list(in_degrees_neighbors.keys()), weights=in_weights, k=10)
            possible_edges = [
                (source, target) for source in sources for target in targets
                if source != target and not net_new.in_edges(source, target)
            ]
            if possible_edges != []:
                new_edge = random.choice(possible_edges)
                n_edges_added += 1
                net_new.add_edge(*new_edge)
                neighborhood_0 = list(net_new.predecessors(new_edge[0])) + list(net_new.successors(new_edge[0]))
                neighborhood_1 = list(net_new.predecessors(new_edge[1])) + list(net_new.successors(new_edge[1]))
                affected_nodes = [new_edge[0], new_edge[1]] + list(set(neighborhood_0).intersection(set(neighborhood_1)))
                for node in affected_nodes:
                    clustering_dict[node] = nx.clustering(net_new, node)
                new_average_clustering = np.average(list(clustering_dict.values()))
                edges_added_clustering += 1
        else:
            # Add new edge based on target degree distribution
            sources_sample = random.choices(list(out_degrees.keys()), weights=out_degrees.values(), k=10)
            targets_sample = random.choices(list(in_degrees.keys()), weights=in_degrees.values(), k=10)
            edge_sample = [
                (source, target) 
                for source in sources_sample 
                for target in targets_sample 
                if source != target and not net_new.has_edge(source, target)]
            if edge_sample != []:
                new_edge = random.choice(edge_sample) # Throws an error if no edges are available
                n_edges_added += 1
                net_new.add_edge(*new_edge)
                neighborhood_0 = list(net_new.predecessors(new_edge[0])) + list(net_new.successors(new_edge[0]))
                neighborhood_1 = list(net_new.predecessors(new_edge[1])) + list(net_new.successors(new_edge[1]))
                affected_nodes = [new_edge[0], new_edge[1]] + list(set(neighborhood_0).intersection(set(neighborhood_1)))
                for node in affected_nodes:
                    clustering_dict[node] = nx.clustering(net_new, node)
                new_average_clustering = np.average(list(clustering_dict.values()))
                edges_added_degree_dist += 1
        # print(f"{n_edges_added=:,} edges added")
    print(f"{edges_added_clustering:,} edges added to increase clustering")
    print(f"{edges_added_degree_dist:,} edges added based on {target_degree_dist} degree distribution")
    return net_new

## Cluster

In [11]:
def get_triangles(G: nx.DiGraph):
    """Return the list of all triangles in a directed graph G."""
    triangles = []
    for clique in nx.enumerate_all_cliques(G.to_undirected()):
        if len(clique) <= 3:
            if len(clique) == 3:
                triangles.append(clique)
        else:
            return triangles
    return triangles

def decluster(net: nx.DiGraph, n_triangles: int) -> nx.DiGraph:
    """
    Decluster the network by rewiring n_triangles random triangles.
    """
    decluster_net = copy.deepcopy(net)
    triangles = get_triangles(net)
    rewired_triangles = random.sample(triangles, n_triangles)
    rewired_edges = [
        (source, target) 
        for (source, target, _) in rewired_triangles
    ] # Warning: triangles are based on undirected graph!
    
    for edge in rewired_edges:
        # Remove edge
        if decluster_net.has_edge(*edge):
            decluster_net.remove_edge(*edge)
        elif decluster_net.has_edge(edge[1], edge[0]):
            decluster_net.remove_edge(edge[1], edge[0]) 
        else:
            continue
        
        # Add new edge based on out- and in-degree distribution
        out_degrees = dict(net.out_degree())
        in_degrees = dict(net.in_degree())
        sources_sample = random.choices(list(out_degrees.keys()), weights=out_degrees.values(), k=10)
        targets_sample = random.choices(list(in_degrees.keys()), weights=in_degrees.values(), k=10)
        edge_sample = [
            (source, target) 
            for source in sources_sample 
            for target in targets_sample 
            if source != target and not decluster_net.has_edge(source, target)]
        new_edge = random.choice(edge_sample) # Throws an error if no edges are available
        decluster_net.add_edge(*new_edge)
    return decluster_net

In [12]:
def cluster_network(net: nx.DiGraph, n: int) -> nx.DiGraph:
    # Create a copy of the original network
    cluster_net = copy.deepcopy(net)
        
    
    # Add edges based on the degree distribution
    n_edges_to_add = n
    print(f"{n_edges_to_add=:,}")

    # Add edges in neighborhoods
    edges_new = []
    while len(edges_new) < n_edges_to_add:
        node = random.choice(list(net.nodes()))
        neighbors = list(net.predecessors(node)) + list(net.successors(node))
        out_degrees_neighbors = dict(net.out_degree(neighbors))
        in_degrees_neighbors = dict(net.in_degree(neighbors))
        out_weights = out_degrees_neighbors.values()
        if all(out_weights) == 0:
            out_weights = np.ones(len(out_degrees_neighbors.keys()))
        in_weights = in_degrees_neighbors.values()
    
        if all(in_weights) == 0:
            in_weights = np.ones(len(in_degrees_neighbors.keys()))
        
        sources = random.choices(list(out_degrees_neighbors.keys()), weights=out_weights, k=10)
        targets = random.choices(list(in_degrees_neighbors.keys()), weights=in_weights, k=10)
        possible_edges = [
            (source, target) for source in sources for target in targets
            if source != target and not (source, target) in edges_new and not net.in_edges(source, target)
        ]
        if possible_edges != []:
            edges_new.append(random.choice(possible_edges))
    cluster_net.add_edges_from(edges_new)
    
    return cluster_net